In [ ]:
# Install conda

%%bash
wget https://repo.anaconda.com/miniconda/Miniconda3-py39_4.12.0-Linux-x86_64.sh
bash Miniconda3-py39_4.12.0-Linux-x86_64.sh
apt update
apt install parallel bcftools

# Install conda
%%bash
wget https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_19/gencode.v19.annotation.gff3.gz
gzip -d gencode.v19.annotation.gff3.gz

In [ ]:
#genes ALS2 ANG CHMP2B DAO DCTN1 FIG4 FUS NEFH OPTN PFN1 PON1 PON2 PON3 PRPH SETX SOD1 SQSTM1 TARDBP TREM2 UBQLN2 VAPB VCPVEGFA

# the below code is to generate the regions of all those genes

%%bash
parallel --gnu " ./GeneTarget/getRegions.sh {} > {}.regions  " ::: ALS2 ANG CHMP2B DAO DCTN1 FIG4 FUS NEFH OPTN PFN1 PON1 PON2   PON3 PRPH SETX  SOD1  SQSTM1 TARDBP TREM2 UBQLN2  VAPB VCPVEGFA

In [ ]:
%%bash
wc -l *regions

In [ ]:
# Download VCF files from the project
%%bash
dx download SV/*

In [ ]:
%%bash

echo "ALS2 ANG CHMP2B DAO DCTN1 FIG4 FUS NEFH OPTN PFN1 PON1 PON2   PON3 PRPH SETX  SOD1  SQSTM1 TARDBP TREM2 UBQLN2  VAPB VCPVEGFA"| tr -s ' ' $'\n' > genes
ls GeneTarget/data/ | grep -P "gz$" > samples
parallel --gnu 'bcftools view -R {1}.regions GeneTarget/data/{2} > events/{1}_{2} ' :::: genes :::: samples                               

In [ ]:
# Generate the table.csv

from os import listdir
from os.path import isfile, join
mypath='events/'

data={}
genes=set()
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
for fileName in onlyfiles:
    i=fileName.find("_")
    gene=fileName[:i]
    sample=fileName[i+1:]
    genes.add(gene)
    lines= [l.strip() for l in open('events/'+fileName)]
    svs=filter(lambda l: len(l)>0 and l[0]!='#',lines)
    numEvents=len(list(svs))
    if sample not in data:
        data[sample]={}
    data[sample][gene]=numEvents

genes=list(genes)
outputFile=open("out.csv",'w')
header=[sample] + genes
outputFile.write(",".join(header)+"\n")

for sample, events in data.items():
    line=[sample]+[str(events[g]) for g in genes]
    outputFile.write(",".join(line)+"\n")
    